# Trabalho Prático 2: Realidade Aumentada

## Alunos: 

- Felipe Eduardo dos Santos - 2017021223
- Renan Antunes Braga Bomtempo - 2018048524

# Dependências e funções auxiliares


In [1]:
# OpenCV
import cv2

# OpenGL
from OpenGL.GL import *
from OpenGL.GLUT import *
from OpenGL.GLU import *

import numpy as np
from objloader import *

# Calcula erro medio quadratico
def calc_rmse(predictions, targets):
    return np.sqrt(((predictions - targets) **2).mean())

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


## Calibração da câmera

Para calibrar a câmera, extraimos 24 frames do video (1 frame de cada segundo do vídeo), e utilizamos as imagens no Toolkit disponibilizado para Octave. Com os parâmetros intrínsecos da câmera em mãos, podemos montar nossa matriz

In [2]:
# Focal length
foc_len = (1203.33680, 1205.25410)

# Principal point
pri_pt  = (959.50000,  539.50000)

# Distortion coefficients
dist_coef = np.array([0.07433, -0.17385, -0.00486, 0.00222, 0.],dtype="float32")

# Camera matrix
camera_matrix = np.array([[foc_len[0],          0., pri_pt[0]],
                          [        0.,  foc_len[1], pri_pt[1]], 
                          [        0.,          0.,        1.]],dtype="float32")

## Configurações da GLUT

In [3]:
# Viewport dimensions
dimensions = (1920, 1080)

# GLUT configuration
glutInit()
glutInitDisplayMode(GLUT_RGBA | GLUT_DOUBLE)
glutSetOption(GLUT_ACTION_ON_WINDOW_CLOSE, GLUT_ACTION_CONTINUE_EXECUTION)
glutInitWindowSize(*dimensions)
window = glutCreateWindow(b'Realidade Aumentada')

# Definindo as funções
## Inicialização dos parâmetros da OpenGL

In [4]:
(width, height) = dimensions

glClearColor(0.0, 0.0, 0.0, 0.0)
glClearDepth(1.0)
glEnable(GL_DEPTH_TEST)

glMatrixMode(GL_PROJECTION)
glLoadIdentity()

fx = foc_len[0]
fy = foc_len[1]

fovy = 2*np.arctan(0.5*height/fy)*180/np.pi
aspect = (width*fy)/(height*fx)
gluPerspective(fovy, aspect, 0.01, 200.0)

glMatrixMode(GL_MODELVIEW)

def idleCallback():
    glutPostRedisplay()

## Encontrando o alvo na cena
Dado um frame do video, queremos encontrar onde estão os alvos (coordenadas das 4 quinas do alvo). Para isso nós:
- 

In [5]:
aux = 0

# Verifica se um candidato a alvo é um alvo, e se sim, qual sua orientecao
def trackerOrient(img, alvo, points,frame):
    global aux
    threshold = 127

    palvo = np.ones(alvo.shape)
    palvoPoints = np.array([[0,0],[0,alvo.shape[1]],
                            [alvo.shape[0],alvo.shape[1]],[alvo.shape[0],0]], dtype="float64")

    h, status = cv2.findHomography(points, palvoPoints)
    palvo = cv2.warpPerspective(img, h, (palvo.shape[0], palvo.shape[1]))

    alvo0 = cv2.cvtColor(alvo, cv2.COLOR_BGR2GRAY)
    _, alvo0 = cv2.threshold(alvo0, threshold, 255, cv2.THRESH_BINARY)

    # Criando os alvos rotacionados
    alvo1 = cv2.rotate(alvo0, cv2.ROTATE_90_CLOCKWISE)
    alvo2 = cv2.rotate(alvo1, cv2.ROTATE_90_CLOCKWISE)
    alvo3 = cv2.rotate(alvo2, cv2.ROTATE_90_CLOCKWISE)


    alvos = [alvo0,alvo1,alvo2,alvo3]

    # Verificando qual orientação produz o menor erro quadratico
    orient = -1
    minErr = 999
    for i in range(4):
        err = calc_rmse(palvo,alvos[i])
        if err<4 and err<minErr:
            minErr = err
            orient = i

    # No vetor de pontos os poontos estão em sentido anti-horario, mas enumerei
    # As possiveis rotções no sentido horario, então esse order inverte isso
    orderAnti = [0,3,2,1]
    orient = orderAnti[orient]

    # Origem
    if orient>=0:
        #origem = order[orient]
        return True, orient

    return False,(0,0)
    #--------------------------------------------------------------------------
    # Marcar na imagem qual a origem estimada
    # order = [points[0][0],points[3][0],points[2][0],points[1][0]]
    # frame = cv2.circle(frame, tuple(order[orient]), 10, (255,0,0), -1)


    aux += 1

def detectMarkers(img):
    # assumindo que o alvo é quadrado
    alvo = cv2.imread('alvo.jpg')
    alvo = cv2.resize(alvo,(322,322),interpolation = cv2.INTER_AREA)

    # Binarização
    threshold = 127

    # Área mínima para ser detectada
    ax = 100*100

    imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _,imgBin = cv2.threshold(imgGray, threshold, 255, cv2.THRESH_BINARY)
    imgCanny = cv2.Canny(imgBin, 100, 200)

    # Dilatar as bordas
    kernel = np.ones((5, 5))
    imgCanny = cv2.dilate(imgCanny, kernel, iterations=1)

    # Encontra os contornos da imagem. RETR_EXTERNAL apenas os mais externos RETR_TREE todos
    contours, _ = cv2.findContours(imgCanny, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    markers = []

    numTrakers = 0
    for contour in contours:
        approx = cv2.approxPolyDP(contour, 0.01 * cv2.arcLength(contour, True), True)

        # Caso a curva tiver 4 lados, for convexa e possui uma area minima de ax pixels
        if len(approx) == 4 and cv2.isContourConvex(approx) and cv2.contourArea(approx)>ax:
            # Ordem em que os vertices foram escritos
            # drawOrder(img, approx)
            istraker, indice_origem = trackerOrient(imgBin, alvo, approx,img)

            if istraker:
                numTrakers += 1
                origem = approx[indice_origem][0]
                eixox = [origem, approx[(indice_origem+1)%4][0]]
                eixoy = [origem, approx[(indice_origem-1)%4][0]]

                approx_fixed = []
                for i in range(4):
                    approx_fixed.append([approx[(indice_origem+i)%4][0][0],approx[(indice_origem+i)%4][0][1]])


                # markers.append((approx, indice_origem))
                markers.append((approx_fixed, indice_origem))
                
                # Desenhar na exixos na tela
                cv2.drawContours(img, [approx], 0, (0, 255, 0), 1)
                img = cv2.circle(img, tuple(origem), 10, (255, 0, 0), -1)

                # Desenhar os eixos
                img = cv2.line(img, tuple(eixox[0]), tuple(eixox[1]), (255,0,0), 6)
                img = cv2.line(img, tuple(eixoy[0]), tuple(eixoy[1]), (0, 0, 255), 6)
                # Colocar nome nos alvos
                org = (approx[0][0][0]-50,approx[0][0][1]-50)
                img = cv2.putText(img, 'Alvo '+str(numTrakers-1), org, cv2.FONT_HERSHEY_SIMPLEX,
                                    1, (255, 0, 0), 2, cv2.LINE_AA)
                
        # Desenhar todos os contornos
        #cv2.drawContours(img, [approx], 0, (0, 0, 255), 1)
    # print("Detected :",numTrakers)
    return markers

## Posicionar o Pikachu em cima do alvo

Com o alvo detectado, e os parâmetros intrínsecos da câmera em mão, podemos encontrar a matriz de projeção que nos permitirá renderizar o modelo 3D em cima do alvo.

In [6]:
def object3D(img_pts, orient, camera_matrix, dist_coef, obj):
    # Marker points in world space
    world_pts = np.array([[-1, -1, 0], [1, -1, 0],
                          [ 1,  1, 0], [-1, 1, 0]], dtype="float32")

    # Marker points in image space
    img_pts = np.array(img_pts, dtype="float32")

    # Calculate rotation and translation vectors
    _, rot_vecs, t_vecs = cv2.solvePnP(world_pts, img_pts, camera_matrix, dist_coef)

    # Generate rotation matrix
    rot_m = cv2.Rodrigues(rot_vecs)[0]

    # Build projection matrix
    proj_matrix = np.array([[rot_m[0][0], rot_m[0][1], rot_m[0][2], t_vecs[0]],
                            [rot_m[1][0], rot_m[1][1], rot_m[1][2], t_vecs[1]],
                            [rot_m[2][0], rot_m[2][1], rot_m[2][2], t_vecs[2]],
                            [        0.0,         0.0,         0.0,      1.0]], dtype="float32")

    # Matrix to convert between OpenCV and OpenGL coordinate systems
    flip_yz = np.array([[1,  0,  0, 0],
                        [0, -1,  0, 0],
                        [0,  0, -1, 0],
                        [0,  0,  0, 1]])

    proj_matrix = np.dot(flip_yz, proj_matrix)
    glLoadMatrixd(np.transpose(proj_matrix))

    # Render model
    glCallList(obj.gl_list)

    # Render wire cube
    #glutWireCube(2.0)

## Renderizar o frame do vídeo como background

Função utilizada para projetar o frame do video como textura em um plano no OpenGL

In [7]:
def drawBackground(frame):
    # Convert frame to OpenGL image "format"
    background = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    background = cv2.flip(background, 0)

    height, width, channels = background.shape
    background = np.frombuffer(background.tobytes(), 
                               dtype=background.dtype, 
                               count=height * width * channels)
    background.shape = (height, width, channels)

    glBindTexture(GL_TEXTURE_2D, background_id)

    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR)
    glTexImage2D(GL_TEXTURE_2D, 0, 3, width, height, 0, GL_RGB, GL_UNSIGNED_BYTE, background)
    glDepthMask(GL_FALSE)

    glMatrixMode(GL_PROJECTION)
    glPushMatrix()
    glLoadIdentity()
    gluOrtho2D(0, width, 0, height)

    glMatrixMode(GL_MODELVIEW)
    glBindTexture(GL_TEXTURE_2D, background_id)
    glTexImage2D(GL_TEXTURE_2D, 0, 3, width, height, 0, GL_RGB, GL_UNSIGNED_BYTE, background)
    glPushMatrix()

    # Plane with video frame texture
    glBegin(GL_QUADS)
    glTexCoord2i(0, 0); glVertex2i(0, 0)
    glTexCoord2i(1, 0); glVertex2i(width, 0)
    glTexCoord2i(1, 1); glVertex2i(width, height)
    glTexCoord2i(0, 1); glVertex2i(0, height)
    glEnd()
    glPopMatrix()

    glMatrixMode(GL_PROJECTION)
    glPopMatrix()

    glMatrixMode(GL_MODELVIEW)
    glDepthMask(GL_TRUE)
    glDisable(GL_TEXTURE_2D)

## Loop de renderização

In [8]:
def displayCallback():
    glMatrixMode(GL_MODELVIEW)
    glLoadIdentity()
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)

    # Habilita o uso de texturas
    glEnable(GL_TEXTURE_2D)

    read, frame = vid.read()

    if read:
        drawBackground(frame)

        # Detect image markers
        markers = detectMarkers(frame)
        
        # For each 
        for marker in markers:
            # Place OBJ model on top of the marker
            object3D(marker[0], marker[1], camera_matrix, dist_coef, obj)
        glutSwapBuffers()    

# Executando o aplicativo

In [9]:
# Open input video
vid = cv2.VideoCapture('tp2-icv-input.mp4')

# Load Pikachu model
obj = OBJ("Pikachu.obj", swapyz=True)

# Background 
background_id = glGenTextures(1)

glutDisplayFunc(displayCallback)
glutIdleFunc(idleCallback)

glutMainLoop()

vid.release()